<a href="https://colab.research.google.com/github/Maharshi-Project/InvestSmart/blob/main/Mandate_2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip3 install unstructured libmagic python-magic

In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [ ]:
urls = [
    "https://www.moneycontrol.com/news/india/fuel-retailers-swap-central-welfare-scheme-hoardings-with-modis-guarantee-at-petrol-pumps-12402651.html",
    "https://www.moneycontrol.com/news/business/ey-financial-services-spearhead-and-veteran-dealmaker-abizer-diwanji-steps-down-12408411.html",
    "https://www.bbc.com/news/technology-68483732",
    "https://example.com/some-other-url",
    "https://www.business-standard.com/industry/news/air-india-records-lowest-punctuality-rate-at-60-19-in-february-shows-data-124030600848_1.html"
]

In [ ]:
import re

def check_url(url):
    moneycontrol_pattern = r'^https:\/\/www\.moneycontrol\.com\/'
    bbc_pattern = r'^https:\/\/www\.bbc\.com\/'
    business_standards = r'^https:\/\/www.business-standard\.com\/'

    if re.match(moneycontrol_pattern, url):
        return "This is a Moneycontrol URL."
    elif re.match(bbc_pattern, url):
        return "This is a BBC News URL."
    elif re.match(business_standards, url):
        return "This is Business Standard News URL."
    else:
        return "This URL does not match either Moneycontrol or BBC News."

# Check each URL and print the message
for url in urls:
    result = check_url(url)
    print(f"{url} : {result}")


https://www.moneycontrol.com/news/india/fuel-retailers-swap-central-welfare-scheme-hoardings-with-modis-guarantee-at-petrol-pumps-12402651.html: This is a Moneycontrol URL.
https://www.moneycontrol.com/news/business/ey-financial-services-spearhead-and-veteran-dealmaker-abizer-diwanji-steps-down-12408411.html: This is a Moneycontrol URL.
https://www.bbc.com/news/technology-68483732: This is a BBC News URL.
https://example.com/some-other-url: This URL does not match either Moneycontrol or BBC News.
https://www.business-standard.com/industry/news/air-india-records-lowest-punctuality-rate-at-60-19-in-february-shows-data-124030600848_1.html: This is Business Standard News URL.


In [ ]:
loader = UnstructuredURLLoader(urls=urls)

In [ ]:
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
type(data[0].page_content)

str

# **Text Splitting**
**Splitting data into chunks can be done in native python but it is a tidious process. Also if necessary, you may need to experiment with various delimiters in an iterative manner to ensure that each chunk does not exceed the token length limit of the respective LLM.
Langchain provides a better way through text splitter classes.**

In [ ]:
text = """Most traditional communication media, including telephone, radio, television, paper mail, and newspapers, are reshaped, redefined, or even bypassed by the Internet, giving birth to new services such as email, Internet telephone, Internet television, online music, digital newspapers, and video streaming websites. Newspaper, book, and other print publishing have adapted to website technology or have been reshaped into blogging, web feeds, and online news aggregators. The Internet has enabled and accelerated new forms of personal interaction through instant messaging, Internet forums, and social networking services. Online shopping has grown exponentially for major retailers, small businesses, and entrepreneurs, as it enables firms to extend their "brick and mortar" presence to serve a larger market or even sell goods and services entirely online. Business-to-business and financial services on the Internet affect supply chains across entire industries."""

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 200,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

In [ ]:
text = data[0].page_content
chunks = r_splitter.split_text(text)

# for chunk in chunks:
#     print((chunk))
#     print("\n")

In [ ]:
len(chunks)

82

# **Text Encoding**

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(chunks)

In [ ]:
vectors.shape

(82, 768)

In [ ]:
vectors

array([[ 0.01320673,  0.00765528, -0.0240848 , ...,  0.00959999,
        -0.00872671, -0.03693042],
       [ 0.01485062,  0.00409342, -0.02708353, ...,  0.01027958,
         0.05401218,  0.00116658],
       [ 0.00928082,  0.04070557, -0.00973405, ...,  0.04211338,
        -0.01261995, -0.01598052],
       ...,
       [-0.01647301,  0.05644056,  0.01641973, ..., -0.02802298,
        -0.05371527, -0.01463401],
       [-0.03151038,  0.05948889, -0.01785636, ..., -0.00921739,
         0.01188063,  0.00720132],
       [-0.0372266 ,  0.07374094,  0.00183831, ...,  0.020676  ,
        -0.01677126,  0.00475836]], dtype=float32)

In [ ]:
dim = vectors.shape[1]
dim

768